In [70]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import warnings

# Hiding warnings
warnings.filterwarnings('ignore')

# File to Load
data_to_load = Path('Resources/raw_data/storm_raw_data.csv')

# Read Data File and store into Pandas DataFrames
raw_data = pd.read_csv(data_to_load)

raw_data.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,202310,25,230,202310,27,551,186682,1145781,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Public reports 7.5 inches at Black Tiger Bay C...,CSV
1,202310,25,230,202310,27,1437,186682,1145783,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Local Police Department relays storm total sno...,CSV
2,202310,25,230,202310,27,1126,186682,1145784,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Public reports 10 inches of storm total snowfa...,CSV
3,202310,25,230,202310,27,1301,186682,1145796,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Emergency Manager reports 6 inches of storm to...,CSV
4,202310,25,230,202310,27,600,186682,1145884,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",CoCoRaHS Station ND-GF-23 reports 8.8 inches o...,CSV


In [71]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75060 entries, 0 to 75059
Data columns (total 51 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   BEGIN_YEARMONTH     75060 non-null  int64  
 1   BEGIN_DAY           75060 non-null  int64  
 2   BEGIN_TIME          75060 non-null  int64  
 3   END_YEARMONTH       75060 non-null  int64  
 4   END_DAY             75060 non-null  int64  
 5   END_TIME            75060 non-null  int64  
 6   EPISODE_ID          75060 non-null  int64  
 7   EVENT_ID            75060 non-null  int64  
 8   STATE               75060 non-null  object 
 9   STATE_FIPS          75060 non-null  int64  
 10  YEAR                75060 non-null  int64  
 11  MONTH_NAME          75060 non-null  object 
 12  EVENT_TYPE          75060 non-null  object 
 13  CZ_TYPE             75060 non-null  object 
 14  CZ_FIPS             75060 non-null  int64  
 15  CZ_NAME             75060 non-null  object 
 16  WFO 

In [72]:
# Removing unnecessary columns
columns_to_keep = ["YEAR", "MONTH_NAME","BEGIN_DAY", "BEGIN_DATE_TIME", "DAMAGE_PROPERTY", "EVENT_TYPE", "STATE", "TOR_F_SCALE", "BEGIN_LOCATION", "BEGIN_LAT", "BEGIN_LON", "EVENT_NARRATIVE"]

filtered_data = raw_data[columns_to_keep]

# Removing rows with missing coordinates and property damage is $0
filtered_data = filtered_data.dropna(subset=["BEGIN_LAT", "DAMAGE_PROPERTY"])


# Ensure "DAMAGE_PROPERTY" is treated as string and strip any whitespace
filtered_data["DAMAGE_PROPERTY"] = filtered_data["DAMAGE_PROPERTY"].astype(str).str.strip()


filtered_data = filtered_data.dropna(subset=["BEGIN_LAT"])
filtered_data = filtered_data[~filtered_data["DAMAGE_PROPERTY"].isin(["0.00K", "0.0"])]


filtered_data.head()

filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12228 entries, 11 to 75059
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YEAR             12228 non-null  int64  
 1   MONTH_NAME       12228 non-null  object 
 2   BEGIN_DAY        12228 non-null  int64  
 3   BEGIN_DATE_TIME  12228 non-null  object 
 4   DAMAGE_PROPERTY  12228 non-null  object 
 5   EVENT_TYPE       12228 non-null  object 
 6   STATE            12228 non-null  object 
 7   TOR_F_SCALE      568 non-null    object 
 8   BEGIN_LOCATION   12228 non-null  object 
 9   BEGIN_LAT        12228 non-null  float64
 10  BEGIN_LON        12228 non-null  float64
 11  EVENT_NARRATIVE  12060 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 1.2+ MB


In [73]:
import pandas as pd

# Sample conversion function (modify as needed)
def convert_damage(damage):
    if isinstance(damage, str):
        damage = damage.replace('$', '').replace(',', '')
        if damage[-1] in ['K', 'M', 'B']:
            factor = {'K': 1_000, 'M': 1_000_000, 'B': 1_000_000_000}[damage[-1]]
            damage = float(damage[:-1]) * factor
        else:
            damage = float(damage)
    return damage


event_types_to_keep = ["Thunderstorm Wind", "Tornado", "Flash Flood", "Flood", "Hail", "Lightning"]
filtered_data = filtered_data[filtered_data['EVENT_TYPE'].isin(event_types_to_keep)]

filtered_data['EVENT_TYPE'] = filtered_data['EVENT_TYPE'].replace('Flash Flood', 'Flood')


# Apply the conversion function to the DAMAGE_PROPERTY column
filtered_data['DAMAGE_PROPERTY'] = filtered_data['DAMAGE_PROPERTY'].apply(convert_damage)

# Now filter based on the converted numeric values
filtered_data = filtered_data[filtered_data['DAMAGE_PROPERTY'] >= 5000]


filtered_data.head()


,YEAR,MONTH_NAME,BEGIN_DAY,BEGIN_DATE_TIME,DAMAGE_PROPERTY,EVENT_TYPE,STATE,TOR_F_SCALE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,EVENT_NARRATIVE
51,2023,July,15,15-JUL-23 17:55:00,7000.0,Thunderstorm Wind,ARKANSAS,NaN,HAMBURG,33.2334,-91.8793,Several trees were blown down.
52,2023,July,15,15-JUL-23 18:10:00,70000.0,Thunderstorm Wind,ARKANSAS,NaN,LAKE VLG,33.3381,-91.2850,Multiple trees and powerlines were blown down ...
59,2023,July,16,16-JUL-23 17:10:00,25000.0,Thunderstorm Wind,MISSISSIPPI,NaN,STAMPLEY,31.6603,-91.1231,Numerous trees were blown down in the county.
60,2023,July,16,16-JUL-23 16:00:00,40000.0,Thunderstorm Wind,LOUISIANA,NaN,JONESVILLE,31.6195,-91.8397,Numerous trees and powerlines were blown down.
67,2023,July,3,03-JUL-23 18:19:00,10000.0,Flash Flood,PENNSYLVANIA,NaN,SHOHOLA,41.4601,-74.9198,"The public reported water, rocks and gravel ru..."


In [74]:
# Checking info
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6325 entries, 51 to 75052
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YEAR             6325 non-null   int64  
 1   MONTH_NAME       6325 non-null   object 
 2   BEGIN_DAY        6325 non-null   int64  
 3   BEGIN_DATE_TIME  6325 non-null   object 
 4   DAMAGE_PROPERTY  6325 non-null   float64
 5   EVENT_TYPE       6325 non-null   object 
 6   STATE            6325 non-null   object 
 7   TOR_F_SCALE      536 non-null    object 
 8   BEGIN_LOCATION   6325 non-null   object 
 9   BEGIN_LAT        6325 non-null   float64
 10  BEGIN_LON        6325 non-null   float64
 11  EVENT_NARRATIVE  6204 non-null   object 
dtypes: float64(3), int64(2), object(7)
memory usage: 642.4+ KB


In [75]:
# Converting BEGIN_DATE_TIME to datetime
filtered_data["BEGIN_DATE_TIME"] = pd.to_datetime(filtered_data["BEGIN_DATE_TIME"], format="%d-%b-%y %H:%M:%S")

# Creating TIME_OF_DAY column
filtered_data["TIME_OF_DAY"] = filtered_data["BEGIN_DATE_TIME"].dt.time


# Reordering columns
columns_order = ["YEAR", "MONTH_NAME","BEGIN_DAY","BEGIN_DATE_TIME","TIME_OF_DAY", "DAMAGE_PROPERTY", "EVENT_TYPE", "STATE", "TOR_F_SCALE", "BEGIN_LOCATION", "BEGIN_LAT", "BEGIN_LON", "EVENT_NARRATIVE"]
filtered_data = filtered_data[columns_order]

filtered_data.head()

,YEAR,MONTH_NAME,BEGIN_DAY,BEGIN_DATE_TIME,TIME_OF_DAY,DAMAGE_PROPERTY,EVENT_TYPE,STATE,TOR_F_SCALE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,EVENT_NARRATIVE
51,2023,July,15,2023-07-15 17:55:00,17:55:00,7000.0,Thunderstorm Wind,ARKANSAS,NaN,HAMBURG,33.2334,-91.8793,Several trees were blown down.
52,2023,July,15,2023-07-15 18:10:00,18:10:00,70000.0,Thunderstorm Wind,ARKANSAS,NaN,LAKE VLG,33.3381,-91.2850,Multiple trees and powerlines were blown down ...
59,2023,July,16,2023-07-16 17:10:00,17:10:00,25000.0,Thunderstorm Wind,MISSISSIPPI,NaN,STAMPLEY,31.6603,-91.1231,Numerous trees were blown down in the county.
60,2023,July,16,2023-07-16 16:00:00,16:00:00,40000.0,Thunderstorm Wind,LOUISIANA,NaN,JONESVILLE,31.6195,-91.8397,Numerous trees and powerlines were blown down.
67,2023,July,3,2023-07-03 18:19:00,18:19:00,10000.0,Flash Flood,PENNSYLVANIA,NaN,SHOHOLA,41.4601,-74.9198,"The public reported water, rocks and gravel ru..."


In [76]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6325 entries, 51 to 75052
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   YEAR             6325 non-null   int64         
 1   MONTH_NAME       6325 non-null   object        
 2   BEGIN_DAY        6325 non-null   int64         
 3   BEGIN_DATE_TIME  6325 non-null   datetime64[ns]
 4   TIME_OF_DAY      6325 non-null   object        
 5   DAMAGE_PROPERTY  6325 non-null   float64       
 6   EVENT_TYPE       6325 non-null   object        
 7   STATE            6325 non-null   object        
 8   TOR_F_SCALE      536 non-null    object        
 9   BEGIN_LOCATION   6325 non-null   object        
 10  BEGIN_LAT        6325 non-null   float64       
 11  BEGIN_LON        6325 non-null   float64       
 12  EVENT_NARRATIVE  6204 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(7)
memory usage: 691.8+ KB


In [77]:
# Export out clean_data.csv

output_path = Path('Resources/clean_data.csv')
filtered_data.to_csv(output_path, index=False)